In [ ]:
#| default_exp uvis

# UVIS
> Data management for Cassini UVIS.

In [ ]:
#| hide
from nbdev.showdoc import show_doc  # noqa

In [ ]:
#| export
import tomlkit
from yarl import URL

from planetarypy.config import config
from planetarypy.pds.opusapi import OPUS
from planetarypy.pds.apps import get_index
from planetarypy.utils import url_retrieve

In [ ]:
#| export
storage_root = config.storage_root / "missions/cassini/uvis"
storage_root

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/uvis')

In [ ]:
get_index("cassini.uvis", "index")

,FILE_NAME,OBSERVATION_TYPE,START_TIME,STOP_TIME,TARGET_NAME,DATA_SET_ID,SPACECRAFT_CLOCK_START_COUNT,SPACECRAFT_CLOCK_STOP_COUNT,INTEGRATION_DURATION,COMPRESSION_TYPE,...,SUB_SOLAR_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SUB_SPACECRAFT_LONGITUDE,PHASE_ANGLE,EMISSION_ANGLE,SOLAR_INCIDENCE_ANGLE,CENTRAL_BODY_DISTANCE,DWELL_TIME,H_LEVEL,D_LEVEL
0,/COUVIS_0001/DATA/D1999_007/EUV1999_007_17_05.LBL,USTARE,1999-01-07 17:05:01.949,1999-01-07 17:08:37.949,<NA>,CO-J-UVIS-2-SPEC-V1.2,1/1294420183.000,UNK,4.0,NONE,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
1,/COUVIS_0001/DATA/D1999_007/EUV1999_007_17_08.LBL,USTARE,1999-01-07 17:08:51.947,1999-01-07 17:12:27.947,<NA>,CO-J-UVIS-2-SPEC-V1.2,1/1294420413.000,UNK,4.0,NONE,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
2,/COUVIS_0001/DATA/D1999_007/FUV1999_007_16_57.LBL,USTARE,1999-01-07 16:57:21.952,1999-01-07 17:00:57.952,<NA>,CO-J-UVIS-2-SPEC-V1.2,1/1294419723.000,UNK,4.0,NONE,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
3,/COUVIS_0001/DATA/D1999_007/FUV1999_007_17_01.LBL,USTARE,1999-01-07 17:01:11.950,1999-01-07 17:04:47.950,<NA>,CO-J-UVIS-2-SPEC-V1.2,1/1294419953.000,UNK,4.0,NONE,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
4,/COUVIS_0001/DATA/D1999_007/HDAC1999_007_16_31...,UHDAC,1999-01-07 16:31:07.962,1999-01-07 16:31:11.962,<NA>,CO-J-UVIS-2-SSB-V1.2,1/1294418149.000,UNK,-999.0,NONE,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,1,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203240,/COUVIS_0060/DATA/D2017_258/FUV2017_258_04_31.LBL,<NA>,2017-09-15 04:31:53.005,2017-09-15 09:04:56.755,SOLAR WIND,CO-S-UVIS-2-CUBE-V1.4,1/1884145299.160,UNK,8191.875,SQRT_9,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
203241,/COUVIS_0060/DATA/D2017_258/FUV2017_258_07_00.LBL,<NA>,2017-09-15 07:00:27.697,2017-09-15 07:13:40.447,SOLAR WIND,CO-S-UVIS-2-SPEC-V1.4,1/1884154214.096,UNK,2.625,SQRT_9,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
203242,/COUVIS_0060/DATA/D2017_258/FUV2017_258_07_15.LBL,<NA>,2017-09-15 07:15:04.442,2017-09-15 10:31:51.692,SOLAR WIND,CO-S-UVIS-2-SPEC-V1.4,1/1884155091.032,UNK,2.625,SQRT_9,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
203243,/COUVIS_0060/DATA/D2017_258/HDAC2017_258_02_14...,<NA>,2017-09-15 02:14:02.433,2017-09-15 02:14:06.433,SOLAR WIND,CO-S-UVIS-2-SSB-V1.4,1/1884137029.000,UNK,-999.0,NONE,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,1,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"


In [ ]:
#| export
class DataManager:

    def __init__(
        self,
        pid: str,  # Product ID. If longer than PDS_ID, will be cut in attribute `pds_id`
        skip_download: bool = False  # skip trying to download
    ):
        self.pid = pid
        self.dict = None
        if not self.raw_data_path.exists() and not skip_download:
            self.download()

    def query(self, pds_id=None):
        pds_id = pds_id if pds_id is not None else self.pds_id
        opus = OPUS(silent=True)
        try:
            self.query_result = opus.query_image_id(pds_id)[0]
        except IndexError:
            raise FileNotFoundError("Project ID not found on PDS server.")
        self.opus_id = self.query_result[0]
        self.dict = self.query_result[1]

    @property
    def pds_id(self):
        return self.pid[:17]

    @property
    def folder(self):
        #         return storage_root / "/".join(self.raw_data_url.parts[4:7])
        return storage_root / self.pds_id

    @property
    def raw_data_url(self):
        if not self.dict:
            self.query()
        return URL(self.dict["couvis_raw"][0])

    @property
    def raw_label_url(self):
        if not self.dict:
            self.query()
        return URL(self.dict["couvis_raw"][1])

    @property
    def raw_data_path(self):
        return self.folder / (self.pds_id + ".DAT")

    @property
    def raw_label_path(self):
        return self.raw_data_path.with_suffix(".LBL")

    @property
    def calib_corr_path(self):
        return self.raw_data_path.with_name(self.raw_data_path.stem + "_CAL_3.DAT")

    @property
    def calib_label_path(self):
        return self.calib_corr_path.with_suffix(".LBL")

    @property
    def original_pid_file(self):
        return self.folder / "original_pid.txt"

    @property
    def results_file(self):
        return self.folder / "urls.toml"

    def download(self, overwrite=False):
        if self.raw_data_path.exists() and not overwrite:
            print("Local files exists. Use `overwrite=True` to download fresh.")
            return
        self.query()
        self.original_pid_file.mk_write(self.pid)
        self.results_file.mk_write(tomlkit.dumps(self.dict))
        self.raw_data_path.parent.mkdir(parents=True, exist_ok=True)
        for key in ["couvis_raw", "couvis_calib_corr"]:
            for url in self.dict[key]:
                url_retrieve(url, self.folder / URL(url).name)

    def __repr__(self):
        s = f"Product ID:\n{self.id}\n\n"
        for k, v in self.query_result[1].items():
            s += f"Key: {k},\nValue(s):\n{v}\n\n"
        return s

In [ ]:
pid = "EUV2002_198_03_26_54_UVIS_C33ST_SPICARAST002_PRIME"

In [ ]:
dm = DataManager(pid)

In [ ]:
dm.raw_label_url

URL('https://opus.pds-rings.seti.org/holdings/volumes/COUVIS_0xxx/COUVIS_0004/DATA/D2002_198/EUV2002_198_03_26.LBL')

In [ ]:
assert dm.pid == pid

PDS data is only stored under the first 17 characters of any UVIS-internal product id.

In [ ]:
assert dm.pds_id == pid[:17]

The originally used product ID `pid` will be stored under `original_pid.txt` in the data folder:

In [ ]:
dm.original_pid_file

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/uvis/EUV2002_198_03_26/original_pid.txt')

The OPUS result file will be stored as a TOML file:

In [ ]:
dm.results_file

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/uvis/EUV2002_198_03_26/urls.toml')

In [ ]:
dm.download(overwrite=False)

Local files exists. Use `overwrite=True` to download fresh.


In [ ]:
dm.raw_data_path

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/uvis/EUV2002_198_03_26/EUV2002_198_03_26.DAT')

In [ ]:
dm.calib_corr_path

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/uvis/EUV2002_198_03_26/EUV2002_198_03_26_CAL_3.DAT')

In [ ]:
#| export
def get_data_path(pid, skip_download=False):
    dm = DataManager(pid, skip_download=skip_download)
    return dm.raw_data_path if dm.raw_data_path.exists() else None


def get_label_path(pid):
    dm = DataManager(pid)
    return dm.raw_label_path

In [ ]:
get_data_path(pid)

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/uvis/EUV2002_198_03_26/EUV2002_198_03_26.DAT')

In [ ]:
get_label_path(pid)

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/uvis/EUV2002_198_03_26/EUV2002_198_03_26.LBL')

In [ ]:
#| export
def get_user_guide():
    url = URL("https://pds-rings.seti.org/cassini/uvis/1-UVIS_Users_Guide_-2018-Jan%2015-For%20PDS-REV-2018-07-06.pdf")
    local_path = storage_root / "uvis_user_guide.pdf"
    if not local_path.exists():
        url_retrieve(url, storage_root / "uvis_user_guide.pdf")
    return local_path

In [ ]:
get_user_guide()

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/cassini/uvis/uvis_user_guide.pdf')